In [1]:
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier

In [2]:
# 读取数据
data = pd.read_csv("/tf/notebooks/facebook-v-predicting-check-ins/train.csv")

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29118021 entries, 0 to 29118020
Data columns (total 6 columns):
row_id      int64
x           float64
y           float64
accuracy    int64
time        int64
place_id    int64
dtypes: float64(2), int64(4)
memory usage: 1.3 GB


In [4]:
a = data.head(10)

In [5]:
a

,row_id,x,y,accuracy,time,place_id
0,0,0.7941,9.0809,54,470702,8523065625
1,1,5.9567,4.7968,13,186555,1757726713
2,2,8.3078,7.0407,74,322648,1137537235
3,3,7.3665,2.5165,65,704587,6567393236
4,4,4.0961,1.1307,31,472130,7440663949
5,5,3.8099,1.9586,75,178065,6289802927
6,6,6.3336,4.3720,13,666829,9931249544
7,7,5.7409,6.7697,85,369002,5662813655
8,8,4.3114,6.9410,3,166384,8471780938
9,9,6.3414,0.0758,65,400060,1253803156


In [6]:
# 处理数据
# 1、缩小数据
data = data.query("x>1.0 & x < 1.25 & y > 2.0 & y<2.25")

In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15469 entries, 4741 to 29117812
Data columns (total 6 columns):
row_id      15469 non-null int64
x           15469 non-null float64
y           15469 non-null float64
accuracy    15469 non-null int64
time        15469 non-null int64
place_id    15469 non-null int64
dtypes: float64(2), int64(4)
memory usage: 846.0 KB


In [8]:
# 处理时间数据
time_value = pd.to_datetime(data['time'], unit='s')
print(time_value.dtype)
time_value = pd.DatetimeIndex(time_value)
time_value

datetime64[ns]


DatetimeIndex(['1970-01-03 00:29:59', '1970-01-08 09:13:29',
               '1970-01-07 14:16:29', '1970-01-07 12:03:09',
               '1970-01-01 20:56:36', '1970-01-07 20:07:34',
               '1970-01-08 18:07:49', '1970-01-08 15:35:38',
               '1970-01-01 09:56:33', '1970-01-08 04:19:16',
               ...
               '1970-01-04 18:42:58', '1970-01-02 15:34:30',
               '1970-01-09 08:20:53', '1970-01-07 09:53:06',
               '1970-01-02 12:02:00', '1970-01-07 00:31:52',
               '1970-01-09 13:25:28', '1970-01-01 04:10:16',
               '1970-01-02 18:37:19', '1970-01-04 23:51:00'],
              dtype='datetime64[ns]', name='time', length=15469, freq=None)

In [9]:
# 构造一些特征
data['day'] = time_value.day
#data.loc[:, "day"] =time_value 
data['hour'] = time_value.hour
data['weekday'] = time_value.weekday

In [10]:
# 删除时间戳特征
data = data.drop('time', axis=1)

In [11]:
data

,row_id,x,y,accuracy,place_id,day,hour,weekday
4741,4741,1.1946,2.0587,54,2538421051,3,0,5
11399,11399,1.1786,2.0303,65,2538421051,8,9,3
19301,19301,1.0211,2.0910,39,4419654085,7,14,2
23951,23951,1.2020,2.0091,67,7341966089,7,12,2
24620,24620,1.1024,2.0448,442,2538421051,1,20,3
...,...,...,...,...,...,...,...,...
29116082,29116082,1.1912,2.0258,81,2538421051,7,0,2
29117586,29117586,1.2220,2.0624,63,2685939991,9,13,4
29117625,29117625,1.1476,2.0215,11,2538421051,1,4,3
29117797,29117797,1.1419,2.1319,8,2842463785,2,18,4


In [15]:
place_count = data.groupby("place_id").count()
place_count

,row_id,x,y,accuracy,day,hour,weekday
place_id,,,,,,,
1008823061,1,1,1,1,1,1,1
1025585791,2,2,2,2,2,2,2
1040557418,1,1,1,1,1,1,1
1068428112,1,1,1,1,1,1,1
1093312343,15,15,15,15,15,15,15
...,...,...,...,...,...,...,...
9929803766,1,1,1,1,1,1,1
9944591314,1,1,1,1,1,1,1
9983648790,4,4,4,4,4,4,4


In [28]:
tf = place_count[place_count.row_id > 3]

tf = tf.reset_index()
tf

,place_id,row_id,x,y,accuracy,day,hour,weekday
0,1093312343,15,15,15,15,15,15,15
1,1110401599,64,64,64,64,64,64,64
2,1121539255,90,90,90,90,90,90,90
3,1181130147,38,38,38,38,38,38,38
4,1195983094,17,17,17,17,17,17,17
...,...,...,...,...,...,...,...,...
257,9700366894,4,4,4,4,4,4,4
258,9766618190,28,28,28,28,28,28,28
259,9846845951,9,9,9,9,9,9,9
260,9903898537,145,145,145,145,145,145,145


In [31]:
data = data[data['place_id'].isin(tf.place_id)]
data
#data = data[data['place_id'].isin(tf.place_id)]

,row_id,x,y,accuracy,place_id,day,hour,weekday
4741,4741,1.1946,2.0587,54,2538421051,3,0,5
11399,11399,1.1786,2.0303,65,2538421051,8,9,3
19301,19301,1.0211,2.0910,39,4419654085,7,14,2
23951,23951,1.2020,2.0091,67,7341966089,7,12,2
24620,24620,1.1024,2.0448,442,2538421051,1,20,3
...,...,...,...,...,...,...,...,...
29116082,29116082,1.1912,2.0258,81,2538421051,7,0,2
29117586,29117586,1.2220,2.0624,63,2685939991,9,13,4
29117625,29117625,1.1476,2.0215,11,2538421051,1,4,3
29117797,29117797,1.1419,2.1319,8,2842463785,2,18,4
